In [1]:
import polars as pl
from catboost import CatBoostRegressor

: 

In [2]:
df_final = pl.read_parquet('../data/train/production_ready_train.parquet')

In [3]:
# split into train and test
X = df_final.drop("IC50 (nM)")
y = df_final["IC50 (nM)"]

In [4]:
def train_test_split_df(df, seed=42, test_size=0.2):
    return df.with_columns(
        pl.int_range(pl.len(), dtype=pl.UInt32)
        .shuffle(seed=seed)
        .gt(pl.len() * test_size)
        .alias("split")
    ).partition_by("split", include_key=False)


def train_test_split(X, y, seed=42, test_size=0.2):
    # Convert Series to DataFrame
    if isinstance(y, pl.Series):
        y = y.to_frame()

    (X_train, X_test) = train_test_split_df(X, seed=seed, test_size=test_size)
    (y_train, y_test) = train_test_split_df(y, seed=seed, test_size=test_size)

    # Convert back to Series if y was originally a Series
    if isinstance(y, pl.DataFrame) and y.shape[1] == 1:
        y_train = y_train.to_series()
        y_test = y_test.to_series()

    return (X_train, X_test, y_train, y_test)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [6]:
X_train = X_train.to_pandas()
X_test = X_test.to_pandas()
y_train = y_train.to_pandas()
y_test = y_test.to_pandas()

In [7]:
param_grid = {
    'iterations': [1000],                   # Number of trees
    'depth': [16],                     # Maximum depth of trees
    'learning_rate': [0.01],           # Step size or learning rate
    'l2_leaf_reg': [7]                       # L2 regularization coefficient
}

In [8]:
# Initialize CatBoost regressor with GPU support and snapshot settings
baseline_model = CatBoostRegressor(task_type='GPU', devices='0')

In [9]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=baseline_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=10)

In [10]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START depth=16, iterations=1000, l2_leaf_reg=7, learning_rate=0.01


: 